In [1]:
import torch
import numpy as np
import os
import json
import codecs
from EduNLP.Pretrain import QuesNetTokenizer, pretrain_quesnet
from EduNLP.Vector import T2V
from EduNLP.I2V import QuesNet, get_pretrained_i2v

os.environ["WANDB_DISABLED"] = "true"

/data/qlh/anaconda3/envs/py39/lib/python3.9/site-packages/js2py/constructors/time_helpers.py:10: UserWarning: Please install or fix tzlocal library (pip install tzlocal) in order to make Date object work better. Otherwise I will assume DST is in effect all the time
  warnings.warn(


# 训练自己的QuesNet模型
## 1. 数据

In [2]:
# 设置你的数据路径和输出路径
BASE_DIR = "../.."

data_dir = f"{BASE_DIR}/static/test_data"
output_dir = f"{BASE_DIR}/examples/test_model/quesnet"

In [3]:
def raw_data():
    _data = []
    data_path = os.path.join(data_dir, "standard_luna_data.json")
    with codecs.open(data_path, encoding="utf-8") as f:
        for line in f.readlines():
            _data.append(json.loads(line))
    return _data

raw_data = raw_data()

## 2. 训练Tokenizer

In [4]:
tokenizer = QuesNetTokenizer(meta=['know_name'], max_length=50,
                             img_dir=os.path.join(data_dir, "quesnet_img"))

# 设置词表
tokenizer.set_vocab(raw_data, key=lambda x: x['ques_content'], trim_min_count=3, silent=False)

print("vocab_size: ", tokenizer.vocab_size)
print()


100%|██████████| 25/25 [00:00<00:00, 134605.39it/s]
[EduNLP, INFO] save meta information know_name: 48


save words(trim_min_count=3): 64/251 = 0.2550                  with frequency 696/929=0.7492
vocab_size:  79



In [5]:
# 保存tokenizer
tokenizer.save_pretrained(output_dir)

## 3. 训练QuesNet

In [7]:
# 自定义训练参数
train_params = {
    # train params
    "n_epochs": 1,
    "batch_size": 1,
    "lr": 1e-3,
    'save_every': 1,
    'log_steps': 10,
    # 'device': 'cpu',
    'max_steps': 2,
    # model params
    'emb_size': 256,
    'feat_size': 256,
}

# 当前仅支持linux下训练
pretrain_quesnet(
    raw_data,
    output_dir,
    # tokenizer=tokenizer,
    img_dir=None,
    save_embs=True,
    load_embs=False,
    train_params=train_params
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1268095/2854330761.py:17 in <module>                                              │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1268095/2854330761.py'                      │
│                                                                                                  │
│ /data/qlh/EduNLP/EduNLP/Pretrain/quesnet_vec.py:759 in pretrain_quesnet                          │
│                                                                                                  │
│   756 │   │   │   **tokenizer_params,                                                            │
│   757 │   │   )                                                                                  │
│   758 │   │   # token_items =                                                                    │
│ ❱ 759 │   │   tokenizer.set_vocab(                                                               │
│   760 │   │   │   items=train_items,                                                             │
│   761 │   │   │   key=data_formation["ques_content"],                                            │
│   762 │   │   │   trim_min_count=data_params.get("trim_min_count", 2),                           │
│                                                                                                  │
│ /data/qlh/EduNLP/EduNLP/Pretrain/quesnet_vec.py:317 in set_vocab                                 │
│                                                                                                  │
│   314 │   │   symbol=None,                                                                       │
│   315 │   │   silent=True,                                                                       │
│   316 │   ):                                                                                     │
│ ❱ 317 │   │   token_items = self.set_text_vocab(                                                 │
│   318 │   │   │   items,                                                                         │
│   319 │   │   │   key=key,                                                                       │
│   320 │   │   │   lower=lower,                                                                   │
│                                                                                                  │
│ /data/qlh/EduNLP/EduNLP/Pretrain/quesnet_vec.py:300 in set_text_vocab                            │
│                                                                                                  │
│   297 │   │   │   the lower bound number for adding a word into vocabulary, by default 1         │
│   298 │   │   silent                                                                             │
│   299 │   │   """                                                                                │
│ ❱ 300 │   │   token_items = self.tokenize(items, key=key, symbol=symbol) if do_tokenize else [   │
│   301 │   │   self.vocab.set_vocab(corpus_items=token_items, trim_min_count=trim_min_count, lo   │
│   302 │   │   self.stoi['word'] = self.vocab.token_to_idx                                        │
│   303 │   │   self.itos['word'] = self.vocab.idx_to_token                                        │
│                                                                                                  │
│ /data/qlh/EduNLP/EduNLP/Pretrain/pretrian_utils.py:299 in tokenize                               │
│                                                                                                  │
│   296 │   │   if isinstance(items, str) or isinstance(items, dict):                              │
│   297 │   │   │   return self._tokenize(items, key=key, **kwargs)                                │
│   298 │   │   else:                                        

## 4. 使用模型

In [7]:
pretrain_dir = os.path.join(output_dir, "quesnet_test_256")

### 4.1 使用训练好的QuesNet Tokenzier

In [8]:
# 读取保存的tokenizer
tokenizer = QuesNetTokenizer.from_pretrained(pretrain_dir,
                                             img_dir=os.path.join(data_dir, "quesnet_img"))

In [9]:
# tokenize
# 可以处理单个题目
print(tokenizer.tokenize(raw_data[0], key=lambda x: x['ques_content']))
print()
# 也可以处理题目列表
print(tokenizer.tokenize(raw_data[:5], key=lambda x: x['ques_content']))

print()

# 将token转换为index
print(tokenizer(raw_data[0], key=lambda x: x['ques_content'], return_text=True, padding=True))
print()
print(tokenizer(raw_data[:3], key=lambda x: x['ques_content'], padding=True))

['已知', '集合', 'A', '=', '\\left', '\\{', 'x', '\\mid', 'x', '^', '{', '2', '}', '-', '3', 'x', '-', '4', '<', '0', '\\right', '\\}', ',', '\\quad', 'B', '=', '\\{', '-', '4', ',', '1', ',', '3', ',', '5', '\\}', ',', '\\quad', 'A', '\\cap', 'B', '=']

[['已知', '集合', 'A', '=', '\\left', '\\{', 'x', '\\mid', 'x', '^', '{', '2', '}', '-', '3', 'x', '-', '4', '<', '0', '\\right', '\\}', ',', '\\quad', 'B', '=', '\\{', '-', '4', ',', '1', ',', '3', ',', '5', '\\}', ',', '\\quad', 'A', '\\cap', 'B', '='], ['复数', 'z', '=', '1', '+', '2', 'i', '+', 'i', '^', '{', '3', '}', '|', 'z', '|', '='], ['埃及', '胡夫', '金字塔', '古代', '世界', '建筑', '奇迹', '形状', '视为', '正四', '棱锥', '以该', '四', '棱锥', '高为', '边长', '正方形', '面积', '等于', '四', '棱锥', '侧面', '三角形', '面积', '侧面', '三角形', '底边', '高', '底面', '正方形', '边长', '比值'], ['设', 'O', '正方形', 'ABCD', '中心', 'O', ',', 'A', ',', 'B', ',', 'C', ',', 'D', '中任取', '3', '点', '取到', '3', '点', '共线', '概率'], ['某校', '课外', '学习', '小组', '研究', '作物', '发芽率', 'y', '温度', 'x', '单位', '^', '{', '\\circ', '}',

### 4.2 使用训练好的QuesNet模型

In [10]:
tokenizer_kwargs = {
    'tokenizer_config_dir': pretrain_dir,
}
i2v = QuesNet('quesnet', 'quesnet', pretrain_dir,
              tokenizer_kwargs=tokenizer_kwargs, device="cpu")

In [11]:
# 获得单个题目的表征
i_vec, t_vec = i2v(raw_data[0], key=lambda x: x["ques_content"])
print(i_vec.shape)
print(t_vec.shape)
print()

# 也可以分别获得题目表征和各个token的表征
t_vec = i2v.infer_token_vector(raw_data[0], key=lambda x: x["ques_content"])
i_vec = i2v.infer_item_vector(raw_data[0], key=lambda x: x["ques_content"])
print(t_vec.shape)
print(i_vec.shape)
print()

# 获得题目列表的表征
t_vec = i2v.infer_token_vector(raw_data[:2], key=lambda x: x["ques_content"])
i_vec = i2v.infer_item_vector(raw_data[:2], key=lambda x: x["ques_content"])
print(t_vec.shape)
print(i_vec.shape)

torch.Size([1, 256])
torch.Size([1, 43, 256])

torch.Size([1, 43, 256])
torch.Size([1, 256])

torch.Size([2, 43, 256])
torch.Size([2, 256])


### 4.3 使用EduNLP中公开的预训练模型

In [12]:
# 获取公开的预训练模型
i2v = get_pretrained_i2v("quesnet_test_256", model_dir=output_dir)

EduNLP, INFO model_path: ..\..\examples\test_model\quesnet\quesnet_test_256
EduNLP, INFO Use pretrained t2v model quesnet_test_256
downloader, INFO http://base.ustc.edu.cn/data/model_zoo/modelhub/quesnet_pub/1/quesnet_test_256.zip is saved as ..\..\examples\test_model\quesnet\quesnet_test_256.zip
downloader, INFO file existed, skipped


In [13]:
# 用法和I2V相同

# 获得单个题目的表征
i_vec, t_vec = i2v(raw_data[0], key=lambda x: x["ques_content"])
print(i_vec.shape)
print(t_vec.shape)
print()

# 也可以分别获得题目表征和各个token的表征
t_vec = i2v.infer_token_vector(raw_data[0], key=lambda x: x["ques_content"])
i_vec = i2v.infer_item_vector(raw_data[0], key=lambda x: x["ques_content"])
print(t_vec.shape)
print(i_vec.shape)
print()

# 获得题目列表的表征
t_vec = i2v.infer_token_vector(raw_data[:2], key=lambda x: x["ques_content"])
i_vec = i2v.infer_item_vector(raw_data[:2], key=lambda x: x["ques_content"])
print(t_vec.shape)
print(i_vec.shape)

torch.Size([1, 256])
torch.Size([1, 43, 256])

torch.Size([1, 43, 256])
torch.Size([1, 256])

torch.Size([2, 43, 256])
torch.Size([2, 256])
